In [13]:
import sys
sys.path.append("../src")
import pandas as pd
from text_cleaning import clean_text
from nltk.tokenize import word_tokenize
from stylographic_features import get_stylographic_feat

from extract_text_feat import style_point_cloud

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
df = pd.read_pickle("../data/csCL2000.pkl")

In [7]:
df.drop(["index", "pdf_link_no_export",
 "path_to_pdf", "pdf_metadata"], axis=1, inplace=True)

In [26]:
# Example of text cleaning
text = df.text_body[0]
text = clean_text(text)

tokenized_text = word_tokenize(text)
len(tokenized_text)

1732

In [45]:
s_vec = style_point_cloud(text, window_size=600, window_overlap=400,
            reduce="pca", redu_kwargs={"n_components": 2})

In [46]:
s_vec.shape

(6, 2)